In [23]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import jieba

In [26]:
test_df = pd.read_csv("/root/mutimedia_final/data/df1.csv")
save_path = "/root/mutimedia_final/result/sentiment_result_2.csv"

In [27]:
test_df['tokenized'] = test_df['微博正文'].apply(lambda x: ' '.join(jieba.cut(x)))
test_df.to_csv('/root/mutimedia_final/result/tokenized_1.csv', index=False) 
print(test_df[['微博正文', 'tokenized']].head())

                                                微博正文  \
0  新浪娱乐:【17-24时要闻回顾】俞灏明回应魏晨结婚时送房：原谅我当年的年少无知http:/...   
1  Cinderellapp月小昱:刚看了一集《探世界》 瞬间穿越回到旧金山 唐人街 干货店 粤...   
2  有新欢如何成功挽回前男友:女生在提出分手的时候，很多人会说对你彻底失望了，我要分手但有些是内...   
3  科技阿康:#iOS13.5正式版推送#为了面对疫情，苹果正式推送了iOS13.5正式版的更新...   
4  泰娱乐论坛:#泰娱乐论坛热门泰剧# 【泰剧《倒霉女孩 / 珍贵的坏运气》人物剧照欣赏（第二部...   

                                           tokenized  
0  新浪 娱乐 : 【 17 - 24 时 要闻 回顾 】 俞灏明 回应 魏晨 结婚 时送 房 ...  
1  Cinderellapp 月 小昱 : 刚看 了 一集 《 探 世界 》   瞬间 穿越 回...  
2  有 新欢 如何 成功 挽回 前男友 : 女生 在 提出 分手 的 时候 ， 很多 人会 说 ...  
3  科技 阿康 : # iOS13.5 正式版 推送 # 为了 面对 疫情 ， 苹果 正式 推送...  
4  泰 娱乐 论坛 : # 泰 娱乐 论坛 热门 泰剧 #   【 泰剧 《 倒霉 女孩   /...  


In [21]:
# Load the trained model and tokenizer
model_path = '/root/mutimedia_final/saved_models/epochs_8' 
model_name = 'bert-base-chinese'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_name)


test_texts = test_df['微博正文'].tolist()

# Tokenize the test data
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, return_tensors="pt")

test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

# Convert to Dataset object
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
})

In [22]:
# Initialize the Trainer
trainer = Trainer(model=model)

# Perform predictions
predictions = trainer.predict(test_dataset)

# Convert predictions to labels (if necessary)
preds = np.argmax(predictions.predictions, axis=1)

# Add predictions to the original dataframe
test_df['predictions'] = preds

# Save the dataframe with predictions
test_df.to_csv(save_path, index=False)  # replace with the actual path to save the predictions

# Print the dataframe with predictions
print(test_df)

/root/anaconda3/envs/mech/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


           微博id         发布者昵称 发布者性别 发布者地区  发布者关注数   发布者粉丝数  \
0     ItwJSk2JW        南郑检察微博     男    陕西     900     3099   
1     ItwJP5Qu0  __Gumusservi     女    上海     107       21   
2     ItwJP5RVK       愉善坊自家果园     女    上海      53       54   
3     ItwJNCk5g          武强视界     男    河北     210     2938   
4     ItwJKkRsP          平安淳化     女    陕西     545     1129   
...         ...           ...   ...   ...     ...      ...   
4856  IqFvigurT         我不是区长     男    浙江    1211    13818   
4857  IqFvc33nz         科学未来人     男    上海     979  1869763   
4858  IqFuKfOJM         财经新媒体     男    北京     411  1831840   
4859  IqFuzB9D8     保定市中级人民法院     女    河北     650   239334   
4860  IqFuzl2np          杨幂翻我     女    其他     714    11624   

                                                   微博正文                 发布时间  \
0     南郑检察微博:【武汉，莫慌！我们共同战疫！】在疫情防控阻击战面前，中国的医务工作者第一时间冲...  2020-02-10 10:59:00   
1     __Gumusservi:#保险杠急救中心[超话]# 打榜抽5000元爱豆代言产品，我正在#...  2020-02-10 10:59:00   
2     愉善坊自家果园:国